In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# List all installed packages and package versions
!pip freeze

In [ ]:
!pip install pycountry_convert

In [ ]:
import pandas as pd
data= pd.read_csv('/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')
print('Data imported Successfully')

In [ ]:
data.columns.to_list()

In [ ]:
[i for i in data.columns if 'Q37' in i]

In [ ]:
data1= data[data['Q5']=='Currently not employed']

In [ ]:
df= data1[[i for i in data.columns if 'Q37' in i]]
df_all= pd.Series(dtype='int')


In [ ]:
for i in df.columns:
    df_all[df[i].value_counts().index[0]]= df[i].count()
    
df_all.reset_index().sort_values(by=0, ascending=False)

In [ ]:
data.head()

In [ ]:
questions= data.iloc[0,:].T


In [ ]:
data= data.iloc[1:, :]

In [ ]:
data['Time from Start to Finish (seconds)'].median()/60

#Hence, a Kaggle user uses on an average 10 min to fill the form.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(data['Time from Start to Finish (seconds)'], shade=True)

#As the data of time to fill form is highely skewed we use median here

In [ ]:
df1= data['Time from Start to Finish (seconds)'].reset_index()


In [ ]:
df1['Time from Start to Finish (seconds)'].apply(lambda x: x/60)

In [ ]:
map= data.Q3.value_counts().to_frame()
map

In [ ]:
import pycountry

map
def countrycode(column):
    Code=[]
    for country in column:
        if country != 'Other':
            try:
                code= pycountry.countries.search_fuzzy(country)[0].alpha_3  #alpha_3 means 3 letter code eg IND
                Code.append(code)
            except:
                Code.append('None')
                
        else:
            Code.append('Other')
    return Code
                


In [ ]:
map['Code']= countrycode(map.index)
map.head()

In [ ]:
map.index

In [ ]:
pycountry.countries.search_fuzzy('India')[0]

In [ ]:
import geopandas

from geopandas import GeoDataFrame

world= geopandas.read_file('/opt/conda/lib/python3.7/site-packages/geopandas/datasets/naturalearth_lowres/naturalearth_lowres.shp')
world.head()

world.columns=['pop_est','continent', 'name', 'Code', 'gdp_md_est','geometry']
world.head()

#Merging the two data frames we get
merge= pd.merge(map, world, on='Code', how ='right')

In [ ]:
geopandas.datasets.get_path('naturalearth_lowres')

In [ ]:
merge= GeoDataFrame(merge).sort_values(by='Q3', ascending=False)

In [ ]:
merge.plot(column='Q3', legend=True,cmap='Reds', figsize=(20,20), scheme='quantiles' , missing_kwds={'color':'grey', 'label': 'Missing Values'})

In [ ]:
data['Q3'].unique()

In [ ]:
data['Q3'].replace({'United States of America':'USA', 'Viet Nam':'Vietnam','Iran, Islamic Republic of...':'Iran', 'United Kingdom of Great Britain and Northern Ireland':'UK'}, inplace=True)

In [ ]:
data['Q3'].unique()

#Renaming the countries and setting inplace=TRUE

In [ ]:
data['Q3'].value_counts()

In [ ]:
import numpy as np
countries = np.asarray(data['Q3'])  #Creating an array for countries

#Creating a dictionary for contients

continents= {
    'NA': 'North America',
    'SA': 'South America',
    'AS': 'Asia',
    'AF': 'Africa',
    'EU': 'Europe',
    'OC': 'Australia',
    'na': 'Others'
    
}

continents

In [ ]:
#coding country tp continent

def continentcode(country):
    try:
        return pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(country))
    except:
        return 'na'
    
    

In [ ]:
import pycountry_convert as pc

In [ ]:
pc.country_name_to_country_alpha2('India')
pc.country_alpha2_to_continent_code('IN')

In [ ]:
pc.convert_continent_code_to_continent_name("AF")

In [ ]:
data1= data.copy()
data1.head()

In [ ]:
data.insert(2, 'continent',[continents[continentcode(country)] for country in countries[:]])


In [ ]:
data.head()

In [ ]:
[continents[continentcode(country)] for country in countries[:]]

In [ ]:
countries[:]

In [ ]:
df_continent= data['continent'].value_counts().reset_index()
df_continent

In [ ]:
ax= plt.bar(df_continent['index'], df_continent['continent'])
ax.set_xlabels(np.asarray(df_continent['index']))

In [ ]:
plt.figure(figsize=(10,10))

ax=sns.barplot(df_continent['index'], df_continent['continent'], palette='summer')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

for p in ax.patches:
    width, height= p.get_width(), p.get_height()
    x, y= p.get_xy()
    ax.annotate("{:.2f}%".format((p.get_height())*100/len(data['continent'])), (x+width/2, y+height/2), ha='center', va='center', fontsize=12)

In [ ]:
for p in ax.patches:
    print(p)
    print(p.get_height())
    print(p.get_xy())

In [ ]:
for p in ax.patches:
    width, height= p.get_width(), p.get_height()
    x, y= p.get_xy()
    ax.annotate(p.get_height()*100/len(data['continent']), (x+width/2, y+height/2))

In [ ]:
len(data['continent'])

In [ ]:
np.asarray(df_continent['index'])

In [ ]:
df_country= data['Q3'].value_counts().reset_index()

In [ ]:
df_country= df_country[:21]

In [ ]:
df_country

plt.figure(figsize=(10,10))
ax= sns.barplot(y= df_country['index'], x= df_country['Q3'], palette='spring')


for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y= p.get_xy()
    ax.annotate("{:.2f}%".format((p.get_width())*100/len(data['Q3'])), (x+width/2, y+height/2), ha='left', va='center')

In [ ]:
for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y= p.get_xy()
    ax.annotate("{:.2f}%".format((p.get_width())*100/len(data['Q3'])), (y+height/2, x+width/2))

In [ ]:
data.head()

In [ ]:
data[data['Q2']=='Nonbinary']

In [ ]:
df_gender= data['Q2'].value_counts().reset_index()
df_gender

In [ ]:
plt.pie(df_gender['Q2'], labels= df_gender['index'], autopct="%1.1f%%")

In [ ]:
import plotly.express as px
px.pie(df_gender, names=df_gender['index'], values=df_gender['Q2'], hole=0)

In [ ]:
data.head(2)

In [ ]:
df_age= data['Q1'].value_counts().reset_index().sort_values(by='index')
df_age

In [ ]:
plt.figure(figsize=(10,10))
ax= sns.barplot(x=df_age['index'], y=df_age['Q1'], palette='magma')

for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y =p.get_xy()
    ax.annotate("{:.2f}%".format(((p.get_height())*100/len(data['Q1']))), (x+width/2, y+height/2), ha='center', va='center', fontsize=12, color='white')


In [ ]:
data.head(2)

In [ ]:
df_education= data['Q4'].value_counts().reset_index()
df_education

In [ ]:
data[data['Q4']=='No formal education past high school']['Q1'].value_counts()

#So most of the people who havent studid beyond hogh school are n 30-34 years old category

In [ ]:
plt.figure(figsize=(10,10))
ax= sns.barplot(x=df_education['index'], y=df_education['Q4'], palette='winter')
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y= p.get_xy()
    ax.annotate("{:.2f}%".format((p.get_height())*100/len(data['Q4'])), (x+width/2,y+height+50), ha='center')

In [ ]:
data[data['Q4']=='Some college/university study without earning a bachelor’s degree']['Q3'].value_counts()

#People who left wothour completinga degree are mostly from India 

In [ ]:
data.head(2)

In [ ]:
data[data['Q3']=='India'].groupby(['Q5']).size().sort_values(ascending=False)

#Finding job propotions in Indian Context 

In [ ]:
df_profession= data['Q5'].value_counts().reset_index()
df_profession

In [ ]:
plt.figure(figsize=(10,10))

ax= sns.barplot(df_profession['index'], df_profession['Q5'], palette='summer')
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=90)

for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y= p.get_xy()
    ax.annotate("{:.2f}%".format((p.get_height())*100/len(data['Q5'])), (x+width/2, y+height+50), ha='center', fontsize='10')

In [ ]:
data.head(2)

In [ ]:
df_exp= data['Q6'].value_counts().reset_index()

df_exp['Percentage']=round(df_exp['Q6']/len(data['Q6'])*100,2)    #round(a,2) is the best option in any case 
df_exp 

In [ ]:
data.columns.to_list()

In [ ]:
df_money=data['Q24'].value_counts().reset_index().sort_values(by='index', ascending=True)

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(y=df_money['index'], x=df_money['Q24'])

In [ ]:
df= data[[i for i in data.columns if 'Q7' in i]]
df_all= pd.Series(dtype='int')
for i in df.columns:
    df_all[df[i].value_counts().index[0]]= df[i].count()
    
df_all.sort_values(ascending=False, inplace=True)

In [ ]:
df_all= df_all.reset_index()

df_all

In [ ]:
plt.figure(figsize=(8,8))

ax= sns.barplot(df_all['index'], df_all[0], palette='magma')
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y= p.get_xy()
    ax.annotate(round((p.get_height())*100/df_all[0].sum(),2), (x+width/2, y+height+50), ha='center')

In [ ]:
df_all[0].sum()

In [ ]:
len(data[[i for i in data.columns if 'Q7' in i]])
data.shape[0]

In [ ]:
df['Q7_Part_1'].value_counts().index[0]
df['Q7_Part_1'].count()

In [ ]:
pd.Series(dtype='int')

In [ ]:
df = data[[i for i in data.columns if 'Q7' in i]]
df_all = pd.Series(dtype='int')
for i in df.columns:
    df_all[df[i].value_counts().index[0]] = df[i].count()
df_all = df_all.sort_values(ascending=True)
df_all

In [ ]:
df_firstuser= data['Q8'].value_counts().reset_index().sort_values(by='Q8',ascending=False)
df_firstuser

In [ ]:
ax= sns.barplot(y=df_firstuser['index'], x=df_firstuser['Q8'], palette='winter')

for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y=p.get_xy()
    ax.annotate(round((p.get_width())*100/df_firstuser['Q8'].sum(),2), (x+width, y+height/2))

In [ ]:
for p in ax.patches:
    print(p)

In [ ]:
df_medium= data['Q11'].value_counts().reset_index()
df_medium

In [ ]:
plt.figure(figsize=(10,10))
ax= sns.barplot(df_medium['index'], df_medium['Q11'])

max_width = 20
ax.set_xticklabels(textwrap.fill(x.get_text(), max_width) for x in ax.get_xticklabels())

In [ ]:
import textwrap

In [ ]:
df= data[[i for i in data.columns if 'Q17' in i]]

df_all= pd.Series(dtype='int')

In [ ]:
for i in df.columns:
    df_all[df[i].value_counts().index[0]]= df[i].count()
    
df_all= df_all.reset_index().sort_values(by=0, ascending=False)

In [ ]:
df_all.head()

In [ ]:
plt.figure(figsize=(10,5))

ax= sns.barplot(df_all['index'], df_all[0])

max_width=8
ax.set_xticklabels(textwrap.fill(x.get_text(), max_width) for x in ax.get_xticklabels())

In [ ]:
df['Q17_Part_1'].value_counts().index[0]

In [ ]:
df= data[[i for i in data.columns if 'Q9' in i]]
df_all= pd.Series(dtype='int')

In [ ]:
for i in df.columns:
    df_all[df[i].value_counts().index[0]]= df[i].count()
    
df_idle= df_all.reset_index()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,8))
df_idle
import seaborn as sns
ax= sns.barplot(df_idle['index'], df_idle[0], palette='hot')

import textwrap

ax.set_xticklabels(textwrap.fill(x.get_text(), 10) for x in ax.get_xticklabels())

for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y= p.get_xy()
    ax.annotate("{:.2f}%".format(p.get_height()*100/df_idle[0].sum()), (x+width/2, y+height), ha='center')

In [ ]:
df_all

In [ ]:
#Currently not employed

data.rename(columns={'Q1':'Age','Q2':'Gender', 'Q3':'Country', 'Q4':'Education', 'Q5':'Current_role',
       'Q6':'Coding_experience', 'Q7_Part_1':'Python_on_regular_basis', 'Q7_Part_2':'R_on_regular_basis', 'Q7_Part_3':'SQL_on_regular_basis','Q7_Part_4':'C_on_regular_basis', 'Q7_Part_5':'C++_on_regular_basis', 'Q7_Part_6':'Java_on_regular_basis',
       'Q7_Part_7':'Javascript_on_regular_basis', 'Q7_Part_8':'Julia_on_regular_basis','Q8':'Programming_lang_recomndtn', 'Q9_Part_1':'Jupyterlab_IDE_on_regular_basis', 'Q9_Part_2':'RStudio_IDE_on_regular_basis',  'Q9_Part_5':'Pycharm_IDE_on_regular_basis',
       'Q9_Part_6':'Spyder_IDE_on_regular_basis', 'Q10_Part_1':'Kaggle_notebook_on_regular_basis', 'Q10_Part_2':'Colab_notebook_on_regular_basis', 'Q11':'Computing_platform', 'Q12_Part_1': 'GPU_specialized_hardware', 'Q12_Part_2':'TPU_specialized_hardware',
       'Q13':'TPU_usage', 'Q14_Part_1':'Matplotlib_visualization_library', 'Q14_Part_2':'Seaborn_visualization_library',
       'Q14_Part_3':'PLotly_visualization_library', 'Q15':'ML_experience','Q16_Part_1':'scikit_learn_framework', 'Q16_Part_2':'Tensorflow_framework', 'Q16_Part_3':'Keras_framework', 'Q16_Part_4':'Pytorch_framework',
       'Q17_Part_1':'lin_log_reg_ML_algorithm', 'Q17_Part_2':'DT_RF_ML_algorithm',
       'Q17_Part_3':'GBoost_ML_algorithm', 'Q17_Part_6':'Neural_network_ML_algorithm', 'Q17_Part_7':'CNN_ML_algorithm',
       'Q17_Part_9':'RNN_ML_algorithm','Q19_Part_1':'Word_embeddings_NLP', 'Q19_Part_2':'Encoder_decoder_NLP','Q20':'Company_size', 'Q21': 'Individuals_num', 'Q22':'ML_implementation_curr_emplyr', 
       'Q24':'Yearly_compensation', 'Q25':'Money_spent_ML', 'Q26_A_Part_1':'AWS_cloud_computing', 'Q26_A_Part_2':'Azure_cloud_computing',
       'Q29_A_Part_1':'MySQL_bigdata', 'Q29_A_Part_2':'PostgreSQL_bigdata','Q30':'Bigdata_products', 'Q31_A_Part_2':'PowerBI_buis_intel','Q31_A_Part_5':'Tableau_buis_intel', 
       'Q32':'Buis_intel_tools', 'Q36_Part_4':'Github_pulicshare', 'Q36_Part_6':'Kaggle_pulicshare', 'Q36_Part_7':'Colab_pulicshare', 'Q37_Part_1':'Coursera_platform', 'Q37_Part_3':'Kaggle_learn_platform', 
       'Q37_Part_7':'Udemy_platform', 'Q37_Part_8':'Linkedin_learn_platform', 'Q37_Part_10':'University_platform','Q38':'Primary_tool', 'Q39_Part_1':'Twitter_media_sources', 'Q39_Part_3':'Reddit_media_sources', 'Q39_Part_4':'Kaggle_media_sources', 'Q39_Part_6':'Youtube_media_sources','Q39_Part_7':'Blogs_media_sources'}, inplace = True)

In [ ]:
data.head()

In [ ]:
df_nemp= data[data['Current_role']=='Currently not employed']

In [ ]:
df_role= data['Current_role'].value_counts().reset_index()
df_role

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
plt.figure(figsize=(10,10))


plt.subplot(2,1,1)
data.groupby('Current_role').size().plot(kind='pie', shadow=True, explode= (0,0.2,0,0,0,0,0,0,0,0,0,0,0,0))


plt.subplot(2,1,2)
px.pie(df_role, names=df_role['index'], values=df_role['Current_role'])


In [ ]:
df_nemp.head(2)

In [ ]:
data.groupby('Current_role').size().reset_index().shape

In [ ]:
import seaborn as sns

plt.figure(figsize=(10,5))
sns.countplot(df_nemp['Age'], hue=df_nemp['Gender'], edgecolor=(0,0,0))
plt.legend(loc=1)



In [ ]:
plt.figure(figsize=(10,5))
ax= sns.countplot(df_nemp['Education'])
import textwrap
ax.set_xticklabels(textwrap.fill(x.get_text(), 10) for x in ax.get_xticklabels())

for p in ax.patches:
    height, width= p.get_height(), p.get_width()
    x,y= p.get_xy()
    
    ax.annotate("{:.2f}%".format((p.get_height())*100/len(df_nemp['Education'])), (x+width/2, y+height+10), ha='center')
    

#Hence most of the people unemployed are with bachelors degree foloowed by masters degree

In [ ]:
plt.figure(figsize=(15,5))
ax= sns.countplot(df_nemp['Country'], palette='magma', edgecolor=(0,0,0))

ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45, )

#So majority of unemployed are from India followed by that from USA

In [ ]:
#Unemployed people have how much experience in the field of amchine learning 
plt.figure(figsize=(10,5))

ax= sns.countplot(df_nemp['Coding_experience'])

ax.set_xticklabels(textwrap.fill(x.get_text(), 10) for x in ax.get_xticklabels())

#Hence, most unemployed are those who have written code for one or less than one year foloowed by people whi have 1-2 years of experience.


#Similar stats apply for those having ML experience.

In [ ]:
#Now we know that most employed are from India, are between age group of 20-25 and have less than 1 year of coding experience.

#Now we find the reason for such unemployement.
plt.figure(figsize=(10,5))
ax= sns.countplot(df_nemp['Education'], hue=df_nemp['Age'], edgecolor=(0,0,0), palette='hot')
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

#Hightest no of Currently Not Employed are in age group 22-24 with bachelors degree and for age group 25-29,it is with people from masters degree

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(df_nemp['ML_experience'], hue=df_nemp['Age'])

#Hence most people unemployed have ML experrience of 1-2 years and are in age range of 22-30

In [ ]:
plt.figure(figsize=(20,5))
sns.countplot(df_nemp['Coding_experience'], hue=df_nemp['ML_experience'], edgecolor=(0,0,0))
plt.legend(loc=1)

#Hence people who have less than 1 year coding exp have less than 1 year ML exp also or have never written a code.
#Similarly people who have 10-2- years exp in coding have written M<L codes for 10-20 years n all.

In [ ]:
[i for i in df_nemp.columns if 'Blogs' in i]

In [ ]:
pd.crosstab(df_nemp['Age'], df_nemp['Blogs_media_sources'])

In [ ]:

pd.crosstab(df_nemp['Age'], df_nemp['Blogs_media_sources']).plot(kind='bar', edgecolor='black', linewidth=2, figsize=(10,5))

#No of people in age group of 22-30 uses blogs to promote their products.  

In [ ]:
pd.crosstab(df_nemp["Age"], df_nemp["Blogs_media_sources"]).plot(kind = 'bar',fontsize = 12,figsize =(18, 7), rot = 0, 
                                                                title = "Age v/s Blogs_media_sources", xlabel="Age", ylabel = "Count", width=.5,color='m',edgecolor='black',linewidth=1.2)

In [ ]:
#Now we see how many unemployed studies through coursera and what is their education status

pd.crosstab(df_nemp['Education'], df_nemp['Coursera_platform']).plot(kind='bar')

In [ ]:
#Finding name of column with coursera

[i for i in df_nemp.columns if 'Coursera' in i ]

In [ ]:
[i for i in df_nemp.columns if 'framework' in i]

In [ ]:
df1= pd.crosstab(df_nemp['ML_experience'],df_nemp['scikit_learn_framework'])
df2= pd.crosstab(df_nemp['ML_experience'], df_nemp['Tensorflow_framework'])
df3= pd.crosstab(df_nemp['ML_experience'], df_nemp['Keras_framework'])

df_frame= df1.merge(df2, on='ML_experience').merge(df3, on='ML_experience')
df_frame

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df_frame.reset_index(inplace=True)

In [ ]:
df_frame

df_frame.plot(kind='bar', edgecolor='black', linewidth=2)\

#People with all years of ML experience first learn Scikit-Learn and then Tensor Flow and then Keras


In [ ]:
[i for i in df_nemp.columns if 'SQL' in i]

In [ ]:
pd.crosstab(df_nemp['Age'],df_nemp['SQL_on_regular_basis'], ).plot(kind='bar')

In [ ]:
pd.crosstab(data['Money_spent_ML'], data['Company_size']).plot(kind='bar', figsize=(10,5), edgecolor=(0,0,0))

In [ ]:
pd.crosstab(data['Company_size'], data['ML_implementation_curr_emplyr']).plot(kind='bar', figsize=(10,5), edgecolor=(0,0,0))

#So most new companies with less employess are exporing on the methods and big companies with most employees already have these methods.

#So those who are currently not employed have good opportunities in companies with 0-49 employees to explore that area and work in that field.

In [ ]:
[i for i in data.columns if 'Company' in i]

In [ ]:
data= data[1:]

In [ ]:
df_country= data['Country'].value_counts()[:6].reset_index().rename(columns={'index':'Country', 'Country':'Count'})
df_country

In [ ]:
[i for i in data.columns if 'Cou' in i]

In [ ]:
Top5_countries = data['Country'].value_counts()[:5].index.tolist()
Top5_countries

In [ ]:
#But we want data of all the rows of those 5 countries

top5= data['Country'].value_counts()[:6].index.to_list()
top5

In [ ]:
data[data['Country'].isin(top5)]

In [ ]:
df_country= data.loc[data['Country'].isin(top5)]

In [ ]:
pd.crosstab(df_country['Country'], df_country['Company_size']).plot(kind='bar', edgecolor=(0,0,0), linewidth=2)
#So current;y not employed have better chance to go in Brazil, India, Russia and Amrica as they have the highest proportion of companies which are worlking on exploring the area of ML

In [ ]:
data[data['Company_size']=='0-49 employees'].groupby('Country').size().reset_index().sort_values(by=0)

In [ ]:
import matplotlib.pyplot as plt
pd.crosstab(df_country['Country'], df_country['ML_implementation_curr_emplyr']).plot(kind='bar', edgecolor=(0,0,0), figsize=(20,8), )
plt.legend(loc=1)

#As in USA most of the employers already have well established methods. the green bar(exploring ML) is highest in India followed by USA and then Japan.

In [ ]:
[i for i in df_country.columns if "ML" in i]

In [ ]:
pd.crosstab(df_country['Country'], df_country['Money_spent_ML']).plot(kind='bar', edgecolor=(0,0,0))

#In countries like India and USa the money spent on ML is not very high as most of the companies are spending below 10000 usd.
#But in Indian, USA and Japan the companies are also spending highly and amount is mostly more tyan 1000 usd. So those not employed also have good options in these sort of countries


In [ ]:
#Opportunities in Different roles of ML
data['Current_role'].value_counts()[:]
main_role=['Data Scientist', 'Software Engineer', 'Data Analyst', 'Research Scientist', 'Machine Learning Engineer','Business Analyst','Statistician']

In [ ]:
[ i for i in data.columns if 'role' in i ]

In [ ]:
df_mainrole= data[data['Current_role'].isin(main_role)]
df_mainrole.head(2)

In [ ]:
pd.crosstab(df_mainrole['Age'], df_mainrole['Current_role']).plot(kind='bar', edgecolor=(0,0,0), figsize=(10,5))

#This shows that if your age is between 20-25 you are more likely to become a data scientist as the most promising thing.

In [ ]:
pd.crosstab(df_mainrole['Current_role'], df_mainrole['Education']).plot(kind='bar')